In [33]:
from data_loader import *
from scipy import stats
import pandas as pd
import numpy as np
data = load_default()

In [31]:
seven = data.bird[(data.bird.index.year == 1970)]
eight = data.bird[(data.bird.index.year == 1980)]
seven['season'] = seven.index.month % 12 // 3 + 1
eight['season'] = eight.index.month % 12 // 3 + 1
seven = seven[((seven.season == 1) | (seven.season == 3))]
eight = eight[((seven.season == 1) | (eight.season == 3))]

/var/folders/kf/j1jq_1qs78n4gd1vczc8vfw80000gn/T/ipykernel_21069/3362176353.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/kf/j1jq_1qs78n4gd1vczc8vfw80000gn/T/ipykernel_21069/3362176353.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/kf/j1jq_1qs78n4gd1vczc8vfw80000gn/T/ipykernel_21069/3362176353.py:6: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/kf/j1jq_1qs78n4gd1vczc8vfw80000gn/T/ipykernel_21069/3362176353.py:6: FutureWarning:

ValueError: cannot reindex on an axis with duplicate labels

In [78]:
from scipy.interpolate import interp2d
from scipy.ndimage import gaussian_filter, rank_filter
from gtda.homology import VietorisRipsPersistence
from plotting import plot_diagram

def heatmap(x, y, s, bins=1000):
    heatmap, xedges, yedges = np.histogram2d(x, y, bins=bins)
    heatmap = gaussian_filter(heatmap, sigma=s)

    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    return heatmap.T, extent

def interp(x, y):
    bounds = [x.min(), x.max(), y.min(), y.max()]
    f = interp2d(x, y, dens, kind="linear")
    x_coords = np.arange(bounds[0],bounds[1])
    y_coords = np.arange(bounds[2],bounds[3])
    Z = f(x_coords,y_coords)
    return Z, bounds



for i, gp in seven.groupby('season'):
    gp.sort_index(inplace=True)
    x = gp.lon.values
    y = gp.lat.values

    # KDE
    positions = np.vstack([x, y])
    kernel = stats.gaussian_kde(positions)
    dens = kernel(positions)


    # Density Estimation
    # heatmap, bounds = heatmap(x, y, 32)    # Gaussian Image Filter
    # heatmap, bounds = interp(x, y)    # 2D interpolation

    # plt.imshow(heatmap.T, extent=bounds, origin='lower')
    # Overlay Raw data
    # plt.scatter(x, y, c='k', s=1)
    # plt.show()

    # PERSIM
    gp['t'] = gp.index - gp.index.min()
    gp.t = gp.t.astype(int) * 1.15741e-14
    X = np.stack([gp.lat.values, gp.lon.values])
    # X = np.vstack([gp.lat.values, gp.lon.values, gp.t.values])
    x = np.expand_dims(X, axis=0)
    display(x)
    homology_dimensions = [0, 1]

    persistence = VietorisRipsPersistence(
        metric="euclidean", homology_dimensions=homology_dimensions, n_jobs=1
    )


    display(persistence.fit_transform_plot(x))

    break

array([[[ 43.75   ,  42.75   ,  43.75   , ...,  38.91667,  25.41667,
          25.41667],
        [-79.41667, -83.08333, -79.41667, ..., -99.58333, -80.41667,
         -80.41667]]])

array([[[0.00000000e+00, 8.80881641e+03, 0.00000000e+00],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]])

In [28]:
bird = data.bird
# center = dict(lat=bird.lat.median(), lon=bird.lon.median())
